In [1]:
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from math import factorial
from itertools import groupby
import itertools

In [2]:
def rough_sets(genome_dataset):
    '''Return the each triplet's generation, 
       positions and rule set'''

    poss_nucs = ['A', 'C', 'G', 'T']
    generation_track_same = {}
    generation_track_diff = {}
    count_same = 0
    count_diff = 0
    for n, gene in enumerate(genome_dataset):
        rule_sets_same = {}
        rule_sets_diff = {}
        # rule_track = np.zeros(24, dtype=float) 
        index_track_same = {}
        index_track_diff = {}
        for m, nuc in enumerate(gene):             
            if nuc == next_gen(n, genome_dataset, m):
                for t, leo in enumerate(gene):
                    if t != m:
                        if leo == next_gen(n, genome_dataset, t):
                            count_same += 1
                            if t in index_track_same.keys():                                
                                index_track_same[t] += update_probs(poss_nucs, 3, leo)
                            else:
                                index_track_same[t] = update_probs(poss_nucs, 3, leo)
                    
                rule_sets_same[m] = index_track_same

            if nuc != next_gen(n, genome_dataset, m):
                # print('here')
                for r, rna in enumerate(gene):
                    if r != m:
                        if rna != next_gen(n, genome_dataset, r):
                            count_diff += 1
                            if r in index_track_diff.keys():                                
                                index_track_diff[r] += update_probs(poss_nucs, 3, rna)
                            else:
                                index_track_diff[r] = update_probs(poss_nucs, 3, rna)

                rule_sets_diff[m] = index_track_diff
                

        generation_track_same[n+1] = rule_sets_same
        generation_track_diff[n+1] = rule_sets_diff

    return generation_track_same, generation_track_diff, count_same, count_diff

def next_gen(current_gen, genome, pos):
    '''Checks there is a next generation
        on which to run the loop and returns
        that positions codon'''
    if current_gen + 1 == len(genome):
        return False
    else:
        return genome[current_gen + 1][pos]

def update_probs(nucleotides, seq_len, pred_triplet):
    '''Updates counts of each triple codon in 
        rule track list'''

    # poss_combinatons = triplet_codon(sorted(itertools.product(nucleotides, repeat=seq_len)))
    # print(poss_combinatons) 
    
    prob_nuc = np.zeros(len(nucleotides), dtype=float)

    for i, nucleotide in enumerate(nucleotides):
        # print(nucleotide)
        if nucleotide == pred_triplet:
            # print('worked')
            prob_nuc[i] = 1
    # print(prob_nuc)

    return prob_nuc

def fasta_to_list(fa):
    '''Converts fasta files into lists'''
    genetic_data = []

    for record in SeqIO.parse(fa, "fasta"):
        genetic_data.append(list(record.seq))

    genetic_sorted = []

    for i in genetic_data:
        if set(i) == {'G', 'T', 'C', 'A'}:
            genetic_sorted.append(i)

    return genetic_sorted

def triplet_codon(A):
    '''Converts list of individual nucleotides into 
        triplet codons'''
    new_list = []
    for fem in A:
        triplet = [ x+y+z for x,y,z in zip(fem[0::3], fem[1::3], fem[2::3]) ]
        new_list.append(triplet)
    return new_list

def split_data(gen_data):
    '''Splits data list into two equal parts'''
    x = []
    y = []
    split_polymerase = int(len(gen_data)/2)
    x = gen_data[:split_polymerase]
    y = gen_data[len(x):]
    # len(x), len(y)

    # x_train, x_test, y_train, y_test = train_test_split(x, y)

    return x, y[:len(x)]

def groupby_len(gene_seqs):
    return [list(g[1]) for g in groupby(sorted(gene_seqs, key=len), len)]

def predict(gen_dict_same, gen_dict_diff, genome_dataset):
    poss_nucs = ['A', 'C', 'G', 'T']

    same_keys = [pos for pos in gen_dict_same[1].keys()]
    diff_keys = [gem for gem in gen_dict_diff[1].keys()]

    next_seq = np.empty(len(genome_dataset[0]), dtype=str)

    for j in same_keys:
        next_seq[j] = genome_dataset[0][j]

    for t in diff_keys:
        next_seq[t] = poss_nucs[np.argmax(gen_dict_diff[1][diff_keys[0]][t])]
        
    return next_seq
    

    

In [21]:
bundi_ebola_y = fasta_to_list('Bundibugyo ebola Human.fa')

In [26]:
for k in range(len(groupby_len(bundi_ebola_y))):
    print(len(groupby_len(bundi_ebola_y)[k]), k)


7 0
7 1
1 2
2 3
7 4
7 5
3 6
2 7
5 8
7 9
7 10


In [48]:
len(groupby_len(bundi_ebola_y)[5])

7

In [39]:
test_1_same, test_1_diff, test_same, test_diff = rough_sets(groupby_len(bundi_ebola_y)[5][:-1])

In [40]:
test_predict = predict(test_1_same, test_1_diff, groupby_len(bundi_ebola_y)[5][:-1])

In [41]:
len(test_predict), len(groupby_len(bundi_ebola_y)[5][0])

(1026, 1026)

In [49]:
count = 0 
for f, j in enumerate(groupby_len(bundi_ebola_y)[5][6]):
    
    # print(j)
    # print(test_predict[f])
    if test_predict[f] == j:
        count += 1


In [54]:
acc = count/len(test_predict)
acc = [acc]

In [56]:
def list_txt(A, filename):
    with open(filename, 'w') as f:
        for exp in A:
            f.write(str(exp))
list_txt(acc, 'Rough set metrics.txt')          